In [1]:
import pnadc #to install, run pip install git+https://github.com/AlanGanem/pnadc
from tqdm import tqdm
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
import ppscore
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

In [2]:
paths_2020 = [*Path('../data/').glob('*2020.csv')]

In [3]:
if not paths_2020:
    pnadc.get_all([2020], path = 'pnadpy/', protocol = 'https')
    paths_2020 = [*Path('../data/').glob('*2020.csv')]

Chaves_PNADC.pdf download is complete!
Chaves_PNADC.pdf download is complete!
Definicao_variaveis_derivadas_PNADC_20200211.pdf download is complete!
Deflatores.zip download is complete!
Unziping pnadpy/Deflatores.zip
Unzip complete
Deflatores.zip download is complete!
Unziping pnadpy/Deflatores.zip
Unzip complete
Dicionario_e_input_20210617.zip download is complete!
Unziping pnadpy/Dicionario_e_input_20210617.zip
Unzip complete
Estrutura_Atividade_CNAE_Domiciliar_2_0.xls download is complete!
Estrutura_Ocupacao_COD.xls download is complete!
Grupamentos_ocupacionais_atividades_PNADC.pdf download is complete!
PNADcIBGE_Deflator_Trimestral.pdf download is complete!
PNADC_012020.zip download is complete!
Unziping pnadpy/PNADC_012020.zip
Unzip complete
Building pnadpy\PNADC_012020.txt
Multiprocessing chunks


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [02:17<00:00,  5.49s/it]


Done!
Saving .csv
pnadpy/PNADC_012020.csv saved!
PNADC_022020.zip download is complete!
Unziping pnadpy/PNADC_022020.zip
Unzip complete
Building pnadpy\PNADC_022020.txt
Multiprocessing chunks


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:11<00:00,  3.76s/it]


Done!
Saving .csv
pnadpy/PNADC_022020.csv saved!
PNADC_032020_20210208.zip download is complete!
Unziping pnadpy/PNADC_032020_20210208.zip
Unzip complete
Building pnadpy\PNADC_032020.txt
Multiprocessing chunks


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [01:10<00:00,  3.71s/it]


Done!
Saving .csv
pnadpy/PNADC_032020.csv saved!
PNADC_042020.zip download is complete!
Unziping pnadpy/PNADC_042020.zip
Unzip complete
Building pnadpy\PNADC_042020.txt
Multiprocessing chunks


100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [01:04<00:00,  3.82s/it]


Done!
Saving .csv
pnadpy/PNADC_042020.csv saved!


In [4]:
def convert_int(df, dtype = int, nan_fill = -1, valid_range = (0,100)):
    '''
    checks the columns that are composed of only integers, if so, cast to integer
    '''
    if df.__class__ == dd.DataFrame:
        return df.map_partitions(convert_int)
    
    for col in df.columns:
        if (df[col].dropna() == df[col].dropna().values.astype(int)).all():
            if not valid_range is None:
                if all(df[col].between(*valid_range)):
                    df[col] = df[col].fillna(nan_fill).astype(dtype)
            else:
                df[col] = df[col].fillna(nan_fill).astype(dtype)
                                                       
    return df

def rename_variables_pnadc(df, pnadc_dictionary_path):    
    return

def create_parquet_pnadc(path, saving_path = 'PNADC/pnadc_compiled.parquet'):
    with ProgressBar():
        data = dd.read_csv(path)
        data = data.repartition('100MB').drop_duplicates()
        data.to_parquet(saving_path)
    return saving_path

In [5]:
#create_parquet_pnadc(path = 'PNADC/*.csv')
data = dd.read_csv(paths_2020)

ValueError: empty urlpath sequence

In [ ]:
len(data)

In [ ]:
#data = data.astype(str)
data = convert_int(data, dtype = 'str')


In [ ]:
rendimentos_cols = [
    'V405012',
    'V405022',
    'V405112',
    'V405122',
    'V405812',
    'V405822',
    'V405912',
    'V405922',    
    'V403322',
    'V403412',
    'V403422',
    'V405812',
    'V403312',
    
]
rendimentos = data[rendimentos_cols].fillna(0).sum(axis = 1).compute()

In [ ]:
print((rendimentos==0).mean())
plt.clf()
sns.distplot(np.log(rendimentos[rendimentos > 0].sample(frac = 0.1)))
plt.show()

In [ ]:
raca = data['V2010']

raca.compute().value_counts(1)

In [ ]:
data.partitions[0].compute()

# Find Confounders

Confounders are good predictors of both the treatment and the observed outcome.
Knowing what are these variables are usefull because in order to remove bias we have to control for confounders

In [ ]:
#filter1 = data['']
data['V403311'].compute().value_counts(1, dropna = False)

In [ ]:
data['V2010'].compute().value_counts(1, dropna = False)

## Filters

In [ ]:
#msk = ~data['V403311'].isna()
msk = data['V2009'].between(18,65) #idade entre 18 e 65 anos

## Predictive Power Scores

In [ ]:
data['V2010'] = data['V2010'].astype(str)

In [ ]:
rendimento_pps_scores = ppscore.predictors(
    data[msk].compute(),
    y = 'V403311'
)

In [ ]:
cor_pps_scores = ppscore.predictors(
    data[msk].compute(),
    y = 'V2010'
)

In [ ]:
rendimento_pps_scores = rendimento_pps_scores.query('ppscore > 0')
rendimento_pps_scores

In [ ]:
cor_pps_scores = cor_pps_scores.query('ppscore > 0')
cor_pps_scores

In [ ]:
cor_pps_scores.merge(rendimento_pps_scores, how = 'inner', on = 'x', suffixes = ('_cor', '_rendimento'))[['x','ppscore_cor', 'ppscore_rendimento']]

In [ ]:
pnadc.query('V405122', 'pnadpy/input_PNADC_trimestral.txt', )

In [61]:
#data = data.sample(0.1)[['V3009A', 'V2010']]